<a href="https://colab.research.google.com/github/flexinJ/flexin/blob/main/Flexin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Downloading required packages
!pip install yfinance
!pip install numpy
!pip install pandas
!pip install hurst
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Import of the necessary modules
from IPython.display import clear_output 
from time import sleep
from tqdm.notebook import tqdm
import pandas as pd
import requests
import sqlite3
import yfinance as yf
from hurst import compute_Hc
import numpy as np

In [3]:
'''conn = sqlite3.connect("flexin_signals.db")
cur = conn.cursor()
cur.execute("CREATE TABLE IF NOT EXISTS store (item TEXT, quantity INTEGER, price REAL)")
conn.commit()
conn.close()
'''

'conn = sqlite3.connect("flexin_signals.db")\ncur = conn.cursor()\ncur.execute("CREATE TABLE IF NOT EXISTS store (item TEXT, quantity INTEGER, price REAL)")\nconn.commit()\nconn.close()\n'

In [4]:
'''!pwd
!ls'''

'!pwd\n!ls'

In [5]:
def telegram(message):

    apiToken = '6183952601:AAFbfrKfZb3iKX6GfYZQnatxo0lg1S4lL8Y'
    chatID = '@flexinflexinbot1'
    apiURL = f'https://api.telegram.org/bot{apiToken}/sendMessage'

    try:
        response = requests.post(apiURL, json={'chat_id': chatID, 'text': message})
        print(response.text)
    except Exception as e:
        print(e)

In [6]:
class Flexin:
  
  def __init__(self, wallet):
    '''
    Symbol of any pair, stock or commodity
    :param symbol: symbol
    '''
    self.wallet = wallet # Self wallet variable

  def run(self):
    
    for symbol in wallet:
      
      # BBs variables
      n, k = 20, 2 # Moving average periods, Factor to shift the bands
      
      # ATR variables
      p = 14 # Period for calculating the ATR (days)
      
      # Verify signals variables
      verifyBuy, verifySell = False, False # Verify signals
      
      print(f'Analyzing {symbol}...', end='\n') # Show the symbol
      
      #------------------------------------------------------------------------- 
      # Week filter - For period => 'max' and interval='1d'
      #-------------------------------------------------------------------------
      w = yf.Ticker(symbol).history(period='max', interval='1d') # Daily data

      # Hurst exponent filter (https://github.com/Mottl/hurst) -----------------
      H = lambda x: compute_Hc(x)[0] # Compute hurst on anonymous lambda func.
      w['Hurst'] = w['Close'].rolling(window=1024).apply(H) # Hurst D avg
      mean_week_hurst = np.mean(w['Hurst'])

      if mean_week_hurst > 0.5:

        # Bollinger Bands ------------------------------------------------------
        w['Standard Deviation'] = w['Close'].rolling(n).std() # Stdv
        w['Middle Band'] = w['Close'].rolling(n).mean() # Middle
        w['Upper Band'] = w['Middle Band'] + w['Standard Deviation']*k # U
        w['Lower Band'] = w['Middle Band'] - w['Standard Deviation']*k # l 
      
        #-----------------------------------------------------------------------
        # Daily filter - For period => 'max' and interval='1d'
        #-----------------------------------------------------------------------
        d = yf.Ticker(symbol).history(period='max', interval='1d') # Daily data
        
        # Hurst exponent filter (https://github.com/Mottl/hurst) ---------------
        H = lambda x: compute_Hc(x)[0] # Compute hurst on anonymous lambda func.
        d['Hurst'] = d['Close'].rolling(window=1024).apply(H) # Hurst D abg
        mean_day_hurst = np.mean(d['Hurst']) # Current daily hurst
        
        if mean_day_hurst > 0.5:

          # Bollinger Bands ----------------------------------------------------
          d['Standard Deviation'] = d['Close'].rolling(n).std() # Stdv
          d['Middle Band'] = d['Close'].rolling(n).mean() # Middle
          d['Upper Band'] = d['Middle Band'] + d['Standard Deviation']*k # U
          d['Lower Band'] = d['Middle Band'] - d['Standard Deviation']*k # l
        
          #---------------------------------------------------------------------
          # Hour filter - For period => '1y' and interval='1h'
          #---------------------------------------------------------------------
          h = yf.Ticker(symbol).history(period='1y', interval='1h') # Hour data

          # Hurst exponent filter (https://github.com/Mottl/hurst)--------------
          H = lambda x: compute_Hc(x)[0] # Compute hurst
          h['Hurst'] = h['Close'].rolling(window=1024).apply(H) # Hurst H avg
          mean_hour_hurst = np.mean(h['Hurst']) # Currente hour hurst
          mean_4hour_hurst = np.mean(h['Hurst'][:-3:])

          if mean_hour_hurst > 0.5 and mean_4hour_hurst > 0.5:

            # Bollinger Bands --------------------------------------------------
            h['Standard Deviation'] = h['Close'].rolling(n).std() # Stdv
            h['Middle Band'] = h['Close'].rolling(n).mean() # Middle
            h['Upper Band'] = h['Middle Band'] + h['Standard Deviation']*k # U
            h['Lower Band'] = h['Middle Band'] - h['Standard Deviation']*k # L

            print('Data collected!', end='\n')
            print('Hurst in persistent behavior 🏂', end='\n')
            
            # BUY/LONG ---------------------------------------------------------
            if w['Close'][-1]>w['Middle Band'][-1] and d['Close'][-1]>d['Middle Band'][-1] and h['Close'][-4]>h['Middle Band'][-4] and h['Low'][-1]>h['Middle Band'][-1] and h['Open'][-1]>h['Middle Band'][-1] and h['Close'][-1]>h['Upper Band'][-1]:

              verifyBuy = True
              title = 'Flexin Signal 🤖☔️' # Title
              price = h['Close'][-1] # Price
              
              # ATR hour -------------------------------------------------------
              h['H-L'] = h['High']-h['Low']
              h['|H-Cp|'] = np.abs(h['High']-h['Close'].shift(1))
              h['|L-Cp|'] = np.abs(h['Low']-h['Close'].shift(1))
              
              # TR - maximum value among these three possibilities
              h['TR'] = h[['|H-Cp|', 'H-L', '|L-Cp|']].max(axis=1)
              smaTRh = h['TR'].rolling(window=p, min_periods=p).mean()[:p]
              restTRh = h['TR'][p:]
              
              # ATR - Average True Range
              h['ATR'] = pd.concat([smaTRh, restTRh]).ewm(alpha=1/p, adjust=False).mean()
              
              ATR = h['ATR'][-1] # ATR
              
              takeProfit = price+(0.95*ATR) # TP
              stopLoss = price-(1.5*ATR) # SL
              # ----------------------------------------------------------------

              if verifyBuy == True:

                telegram(f'{title}\n{symbol} at price ${price.round(4)}\n📈 BUY\nTP: ${takeProfit.round(4)}\nSL: ${stopLoss.round(4)}') # Show buy info
                sleep(1)
                telegram(f'Opening order...💤')
                sleep(1)
                telegram(f'Open order ✔️')
                sleep(1)
                
                time = 0

                while True:

                  time += 1
                  sleep(1)
                  
                  if h['Close'][-1] < stopLoss:

                    telegram('Price around stop loss 😭')
                    
                    break
                  
                  elif h['Close'][-1] > takeProfit:

                    telegram('Price arround take profit now 🙏💸')
                    
                    break

                  else:

                    continue

                time = 0

            # SELL/SHORT -------------------------------------------------------
            elif w['Close'][-1]<w['Middle Band'][-1] and d['Close'][-1]<d['Middle Band'][-1] and h['Close'][-4]<h['Middle Band'][-4] and h['Low'][-1]<h['Middle Band'][-1] and h['Open'][-1]<h['Middle Band'][-1] and h['Close'][-1]<h['Lower Band'][-1]:
                         
              verifySell = True
              title = 'Signal 🤖☔️' # Title
              price = h['Close'][-1] # Price

              # ATR hour -----------------------------------------------
              h['H-L'] = h['High']-h['Low']
              h['|H-Cp|'] = np.abs(h['High']-h['Close'].shift(1))
              h['|L-Cp|'] = np.abs(h['Low']-h['Close'].shift(1))
            
              # TR - maximum value among these three possibilities
              h['TR'] = h[['|H-Cp|', 'H-L', '|L-Cp|']].max(axis=1)
              smaTRh = h['TR'].rolling(window=p, min_periods=p).mean()[:p] 
              restTRh = h['TR'][p:] # Rest of indicator
              
              # ATR - Average True Range
              h['ATR'] = pd.concat([smaTRh, restTRh]).ewm(alpha=1/p, adjust=False).mean() # ATR

              ATR = h['ATR'][-1] # ATR

              takeProfit = price-(0.99*ATR) # TP
              stopLoss = price+(1.5*ATR) # SL
              # --------------------------------------------------------

              if verifySell == True:

                telegram(f'{title}\n{symbol} at price ${price.round(4)}\n📉 SELL\nTP: ${takeProfit.round(4)}\nSL: ${stopLoss.round(4)}') # Show sell info
                sleep(1)
                telegram(f'Opening order...💤')
                sleep(1)
                telegram(f'Open order ✔️')
                sleep(1)
                
                time = 0

                while True:

                  time += 1
                  sleep(1)
                  
                  if h['Close'][-1] > stopLoss:

                    telegram('Price around stop loss 😭')
                    
                    break
                  
                  elif h['Close'][-1] < takeProfit:

                    telegram('Price arround take profit now 🙏💸')
                    
                    break

                  else:

                    continue

                time = 0

            else:
              
              print(f'No signal for {symbol}', end='\n') # No signal

          else:
          
            print('Hurst in anti-persistent behavior 💤', end='\n') # Possible regression

In [7]:
wallet = ['JPY=X',
          'NZDUSD=X',
          'AUDUSD=X',
          'CAD=X',
          'GBPUSD=X',
          'EURUSD=X']

In [ ]:
while True:
  flexin = Flexin(wallet)
  run = flexin.run()
  clear_output()

Analyzing JPY=X...
Data collected!
Hurst in persistent behavior 🏂
No signal for JPY=X
Analyzing NZDUSD=X...
